In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

In [2]:
# cudaが使えるか確認
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

データの読み込み

GPUで作ったPickleのファイルは，CPUではそのままでは使えないため，工夫が必要．

https://www.kunita-gamefactory.com/post/%E3%80%90pytorch%E3%80%91gpu%E3%81%A7%E8%A8%93%E7%B7%B4%E3%81%95%E3%81%9B%E3%81%9F%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92cpu%E3%81%A7%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%82%82%E3%81%86%E3%81%A8%E3%81%97%E3%81%9F%E3%81%8A%E8%A9%B1

を真似したらうまくCPU上でもファイルを読み込むことができるようになった．

In [3]:
import io
        
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [4]:
# ファイルの相対パスを指定
file_path = '../data storage/Ising_data_L16_v3.pkl'

if torch.cuda.is_available():
    with open(file_path, 'rb') as file:
        loaded_data = pickle.load(file)
else:
    with open(file_path, 'rb') as file:
        loaded_data = CPU_Unpickler(file).load()
        
# 読み込んだデータを個々の変数に分割
spin_data, label_data = loaded_data

In [5]:
# # 磁化を計算
# def magnetization(state):
#     return np.mean(state)

# for i in range(len(spin_data)):
#     mag = magnetization(spin_data[i])
#     if mag > 0:
#         spin_data[i] *= -1

In [6]:
# データのリストをNumPy配列に変換
spin_data_np = np.array(spin_data)
label_data_np = np.array(label_data)

# NumPy配列をPyTorchテンソルに変換
spin_data_tensor = torch.from_numpy(spin_data_np)
label_data_tensor = torch.from_numpy(label_data_np)

In [7]:
# サンプルデータを訓練用とテスト用に分割(5:5)
spin_train, spin_test, label_train, label_test = train_test_split(spin_data_tensor, label_data_tensor, test_size=0.5)

In [8]:
# PyTorchのテンソルに変換
spin_train = torch.tensor(spin_train, dtype=torch.float32)
spin_test = torch.tensor(spin_test, dtype=torch.float32)
label_train = torch.tensor(label_train, dtype=torch.float32)  
label_test = torch.tensor(label_test, dtype=torch.float32)    

C:\Users\soken\AppData\Local\Temp\ipykernel_15912\168836570.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spin_train = torch.tensor(spin_train, dtype=torch.float32)
C:\Users\soken\AppData\Local\Temp\ipykernel_15912\168836570.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spin_test = torch.tensor(spin_test, dtype=torch.float32)
C:\Users\soken\AppData\Local\Temp\ipykernel_15912\168836570.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_train = torch.tensor(label_train, dtype=torch.float32)
C:\Users\soken\AppData\Local\Te

正解データはone-hot表現にする必要がある

In [9]:
# テンソルを新しいテンソルに変換する関数を定義
def to_one_hot(data, num_classes=20):
    # one-hotベクトルの初期化
    one_hot = torch.zeros(len(data), num_classes)
    print(one_hot.size())
    # 各要素を20次元のone-hotベクトルに変換
    for i, val in enumerate(data):
        index = int(torch.round((val - 0.05) / 0.05))
        one_hot[i, index] = 1.0
    
    return one_hot

# label_train,temp_testをone-hotベクトルに変換
one_hot_label_train = to_one_hot(label_train, num_classes=20)
one_hot_label_test = to_one_hot(label_test, num_classes=20)

torch.Size([20000, 20])
torch.Size([20000, 20])


In [10]:
# データセットの作成
train_dataset = TensorDataset(spin_train, one_hot_label_train)
test_dataset = TensorDataset(spin_test, one_hot_label_test)

In [11]:
# 前処理を定義
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
])

# データセットに前処理を適用
transformed_train_dataset = [(transform(tensor_sample), label) for tensor_sample, label in train_dataset]
transformed_test_dataset = [(transform(tensor_sample), label) for tensor_sample, label in test_dataset]

In [12]:
# DataLoaderの設定
train_loader = DataLoader(transformed_train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(transformed_test_dataset, batch_size=20)

モデルを定義

In [13]:
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size, bias=True)
        self.fc2 = nn.Linear(hidden_size, output_size, bias=True)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.flatten(x)
        x = F.softmax(self.fc1(x), dim=1)
        x = F.softmax(self.fc2(x), dim=1)
        return x

モデルのインスタンス化

In [14]:
input_size = 16*16
hidden_size = 3
output_size = 20
model = FCNN(input_size, hidden_size, output_size)
model.to(device)
# モデルの概要表示
print(model)

FCNN(
  (fc1): Linear(in_features=256, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=20, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


損失関数と最適化アルゴリズムの設定

In [15]:
criterion = nn.CrossEntropyLoss()   # クロスエントロピー誤差
optimizer = optim.Adam(model.parameters(), lr=0.0015)     # Adam,L2正則化{, weight_decay=5e-4}

学習の実行

In [16]:
num_epochs = 30
train_losses = []
train_accs = []
test_losses = []
test_accs = []
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        running_loss += loss.item()
        pred = torch.argmax(output, dim=1)      
        targets = torch.argmax(targets, dim=1) 
        running_acc += torch.mean(pred.eq(targets).float().cpu()) 
        optimizer.step()
    running_loss /= len(train_loader)   
    running_acc /= len(train_loader)    
    train_losses.append(running_loss)
    train_accs.append(running_acc)
    #
    #   test loop
    #
    test_running_loss = 0.0
    test_running_acc = 0.0
    for test_inputs, test_targets in test_loader:
        test_inputs = test_inputs.to(device)
        test_targets = test_targets.to(device)
        test_output = model(test_inputs)
        test_loss = criterion(test_output, test_targets)
        test_running_loss += test_loss.item()
        test_pred = torch.argmax(test_output, dim=1)      
        test_targets = torch.argmax(test_targets, dim=1)  
        test_running_acc += torch.mean(test_pred.eq(test_targets).float().cpu()) 
    test_running_loss /= len(test_loader)   
    test_running_acc /= len(test_loader)    
    test_losses.append(test_running_loss)
    test_accs.append(test_running_acc)
        
    print("epoch: {}, loss: {}, acc: {}, test loss: {}, test acc: {}".format(epoch, running_loss, running_acc, test_running_loss, test_running_acc))

epoch: 0, loss: 2.982114176750183, acc: 0.08269992470741272, test loss: 2.966054104804993, test acc: 0.10350003838539124
epoch: 1, loss: 2.958863051176071, acc: 0.10290005058050156, test loss: 2.954762621164322, test acc: 0.09805005043745041
epoch: 2, loss: 2.9522410447597505, acc: 0.10200002789497375, test loss: 2.951401739835739, test acc: 0.0979500487446785
epoch: 3, loss: 2.9496987159252166, acc: 0.10230010002851486, test loss: 2.949986057281494, test acc: 0.09860002994537354
epoch: 4, loss: 2.9483461742401125, acc: 0.10240011662244797, test loss: 2.949053111076355, test acc: 0.09930005669593811
epoch: 5, loss: 2.947365880727768, acc: 0.10440010577440262, test loss: 2.9483068642616272, test acc: 0.09990005940198898
epoch: 6, loss: 2.946365038871765, acc: 0.10835004597902298, test loss: 2.947600288391113, test acc: 0.10235005617141724
epoch: 7, loss: 2.9453415706157684, acc: 0.11315014213323593, test loss: 2.9467869360446928, test acc: 0.1078500896692276
epoch: 8, loss: 2.9442682061

モデルの保存

In [17]:
Ising_size = int(np.sqrt(input_size))
print(Ising_size)
class_name = type(model).__name__
print(class_name)
print(hidden_size)

file_path = f'../data storage/prm_data_L{Ising_size}_{class_name}_Nh{hidden_size}.pth'

torch.save(model.state_dict(), file_path)

16
FCNN
3
